In [1]:
import pandas as pd
import xarray as xr
import ocf_blosc2

In [2]:
# power data
wind_data_ds = xr.open_dataset('/mnt/storage_ssd_4tb/india_wind_data.nc')
wind_data_ds

/home/zak/.local/lib/python3.10/site-packages/xarray/backends/plugins.py:80: RuntimeWarning: Engine 'cfgrib' loading failed:
Could not load the ecCodes library!
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.Dataset>
Dimensions:   (time_utc: 163680)
Coordinates:
  * time_utc  (time_utc) datetime64[ns] 2019-03-31T18:45:00 ... 2023-11-30T18...
Data variables:
    0         (time_utc) float64 ...

In [80]:
merged_data = wind_data_ds.expand_dims(time_utc=['ts'])
merged_data

ValueError: Dimension time_utc already exists.

In [3]:
# Generation metadata
wind_metadata_df = pd.read_csv('/mnt/storage_ssd_4tb/india_wind_metadata2.csv')
wind_metadata_df

,system_id,latitude,longitude,capacity_megawatts
0,0,26.3499,72.6,3381.0


In [4]:
# Modificaiton of the latitude and longitude
wind_metadata_df['latitude'] = 27.0
wind_metadata_df['longitude'] = 71.0
wind_metadata_df

,system_id,latitude,longitude,capacity_megawatts
0,0,27.0,71.0,3381.0


In [58]:
# Nwp data
india_zarr = xr.open_zarr("/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20200113T1200.zarr.zip")
india_zarr

<xarray.Dataset>
Dimensions:         (variable: 17, init_time: 1, step: 85, latitude: 221,
                     longitude: 221)
Coordinates:
  * init_time       (init_time) datetime64[ns] 2020-01-13T12:00:00
  * latitude        (latitude) float64 31.0 30.95 30.9 30.85 ... 20.1 20.05 20.0
  * longitude       (longitude) float64 68.0 68.05 68.1 ... 78.9 78.95 79.0
  * step            (step) timedelta64[ns] 00:00:00 01:00:00 ... 3 days 12:00:00
  * variable        (variable) object 'dlwrf' 'dswrf' 'duvrs' ... 'v100' 'v200'
Data variables:
    ECMWF_NW-INDIA  (variable, init_time, step, latitude, longitude) float32 dask.array<chunksize=(17, 1, 85, 110, 110), meta=np.ndarray>

In [66]:
# example pv dataformat for model
PV_TARGET_DATA_PATH = "/mnt/storage_b/data/ocf/solar_pv_nowcasting/clients/island/pv_hourly_v6.nc"
pv_target_data = xr.open_dataset(PV_TARGET_DATA_PATH)
pv_target_data

<xarray.Dataset>
Dimensions:    (pv_id: 1, ts: 53988)
Coordinates:
  * pv_id      (pv_id) object '0'
  * ts         (ts) datetime64[ns] 2016-12-31T23:00:00 ... 2023-02-28T22:00:00
    latitude   (pv_id) float64 ...
    longitude  (pv_id) float64 ...
Data variables:
    power      (pv_id, ts) float64 ...
    capacity   (pv_id, ts) float64 ...

In [5]:
# Data processing to turn the wind data into the same format as the pv data
wind_data_ds = wind_data_ds.rename_vars({'0': 'power'})

wind_data_ds = wind_data_ds.expand_dims({"pv_id": [wind_metadata_df['system_id'][0]]})

# Assign the attributes directly to wind_data_ds instead of creating a new dataset
wind_data_ds = wind_data_ds.assign_coords({
    'latitude': ('pv_id', [wind_metadata_df['latitude'][0]]),
    'longitude': ('pv_id', [wind_metadata_df['longitude'][0]]),
    'capacity': ('pv_id', [wind_metadata_df['capacity_megawatts'][0]])
})

# wind_data_ds = wind_data_ds.rename_vars({'ss_id': 'pv_id'})
wind_data_ds = wind_data_ds.rename({'time_utc':'ts'})

In [6]:
wind_data_ds

<xarray.Dataset>
Dimensions:    (pv_id: 1, ts: 163680)
Coordinates:
  * pv_id      (pv_id) int64 0
  * ts         (ts) datetime64[ns] 2019-03-31T18:45:00 ... 2023-11-30T18:30:00
    latitude   (pv_id) float64 27.0
    longitude  (pv_id) float64 71.0
    capacity   (pv_id) float64 3.381e+03
Data variables:
    power      (pv_id, ts) float64 346.0 334.0 322.0 302.0 ... 636.0 646.0 631.0

In [7]:
wind_data_ds.to_netcdf('/mnt/storage_ssd_4tb/india_wind_data_format_for_site_model_v6_updated_latlong.nc')

In [56]:
import glob
import os
base_path = "/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/"
zarr_files = glob.glob(os.path.join(base_path, "*.zarr.zip"))

ECMWF_PATH = zarr_files

In [57]:
ECMWF_PATH

['/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20190921T1200.zarr.zip',
 '/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20230220T0000.zarr.zip',
 '/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20230514T1200.zarr.zip',
 '/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20210528T0000.zarr.zip',
 '/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20221026T1200.zarr.zip',
 '/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20200822T0000.zarr.zip',
 '/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20201104T0000.zarr.zip',
 '/mnt/storage_b/data/ocf/solar_pv_nowcasting/nowcasting_dataset_pipeline/NWP/ECMWF/nw-india/zarr/20220906T0000.zarr.zip',
 '/mnt/storage_b

In [94]:
# Look at the results

results_df = pd.read_csv("/home/zak/pv-site-prediction/exp_results/wind_india_v3_4096s/test_errors.csv.gz")

In [97]:
results_df.head(20)

,pv_id,ts,ts_start,ts_end,metric,error,horizon,y,pred,train_date
0,0,2022-12-01 00:00:00,2022-12-01 00:00:00,2022-12-01 00:15:00,mae,646.000000,0,646.0,0.000000,2022-11-30
1,0,2022-12-01 00:00:00,2022-12-01 00:15:00,2022-12-01 00:30:00,mae,648.000000,15,648.0,0.000000,2022-11-30
2,0,2022-12-01 00:00:00,2022-12-01 00:30:00,2022-12-01 00:45:00,mae,633.000000,30,633.0,0.000000,2022-11-30
3,0,2022-12-01 00:00:00,2022-12-01 00:45:00,2022-12-01 01:00:00,mae,622.000000,45,622.0,0.000000,2022-11-30
4,0,2022-12-01 00:00:00,2022-12-01 01:00:00,2022-12-01 01:15:00,mae,593.000000,60,593.0,0.000000,2022-11-30
5,0,2022-12-01 00:00:00,2022-12-01 01:15:00,2022-12-01 01:30:00,mae,584.000000,75,584.0,0.000000,2022-11-30
6,0,2022-12-01 00:00:00,2022-12-01 01:30:00,2022-12-01 01:45:00,mae,579.000000,90,579.0,0.000000,2022-11-30
7,0,2022-12-01 00:00:00,2022-12-01 01:45:00,2022-12-01 02:00:00,mae,330.313543,105,599.0,268.686457,2022-11-30
8,0,2022-12-01 00:00:00,2022-12-01 02:00:00,2022-12-01 02:15:00,mae,346.259032,120,599.0,252.740968,2022-11-30
9,0,2022-12-01 00:00:00,2022-12-01 02:15:00,2022-12-01 02:30:00,mae,240.757979,135,556.0,315.242021,2022-11-30


In [98]:
import numpy as np

average_error = np.mean(results_df['error'])

In [99]:
average_error

350.1834452473457

In [ ]:
# Compare error to this: https://analysis.quartz.energy/
